In [1]:
from lib import Models, Datasets, LossCalculator, Train
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


2025-02-18 20:26:38.910722: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 20:26:38.926560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739899598.943469  296427 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739899598.948592  296427 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 20:26:38.967175: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# model, tokenizer = Models.LoadLLM("distilgpt2")
# model, tokenizer = Models.LoadLLM("meta-llama/Llama-3.2-1B")
model = Models.LoadLLM("meta-llama/Llama-3.2-1B-Instruct")

math = Datasets.LoadMath()

print(model['model'])


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [3]:
# LossCalculator.CalcLoss(**math, max_length=1024, count=10, model=model, tokenizer=tokenizer)

In [4]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'up_proj', 'down_proj'])
# peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['c_attn', 'c_proj', 'c_fc', 'c_proj'])


model['model'] = get_peft_model(model['model'], peft_config)

print(model['model'])


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features=2048

In [5]:
Train.Trains(**math, max_length=1024, count=150, **model)

Running tokenizer on dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

/home/parshina/workspace/AdaptorsResearch/lib/Train.py:113: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,5.724100
20,3.944400
30,2.558100
40,1.652100
50,1.075000
60,0.653800
70,0.540900
80,0.520400
90,0.513500
100,0.507500


TrainOutput(global_step=150, training_loss=1.3458857536315918, metrics={'train_runtime': 86.5393, 'train_samples_per_second': 1.733, 'train_steps_per_second': 1.733, 'total_flos': 904823989862400.0, 'train_loss': 1.3458857536315918, 'epoch': 1.0})

In [6]:
# LossCalculator.CalcLoss(**math, max_length=1024, count=5, **model)

model_path = "./logs/test_new/checkpoint-200"  # Путь к папке с сохраненной моделью
from transformers import AutoTokenizer, AutoModelForCausalLM

t = AutoTokenizer.from_pretrained(model_path)
m = AutoModelForCausalLM.from_pretrained(model_path)

In [7]:
from transformers import pipeline

dirc = "logs/test_new/checkpoint-200"
# generator = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct")
# generator = pipeline("text-generation", model=dirc, tokenizer=dirc)
# generator = pipeline("text-generation", model=m, tokenizer=t)
generator = pipeline("text-generation", model=model['model'], tokenizer=model['tokenizer'])



messages = [
    {"role": "user", "content": "Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\nCompute $\dbinom{16}{5}$."},
]
messages = [
    {"role": "user", "content": "I hate cats!!!!!"},
]

generated_text = generator(messages,  max_length=500, num_return_sequences=1)

for i, text in enumerate(generated_text):
    print(f"Generated text {i + 1}:")
    text = text['generated_text']
    for entry in text:
        if entry['role'] == 'assistant':
            print(entry['content'])


Device set to use cuda
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'Jamba

Generated text 1:
You are so stupid!!!! Cats are so cute!!!!!! Do you like dogs!!!!!!)


In [8]:
I love cats! Hahaha 
I - 345
 lo - 1000


1 2 3 4 5
1 2 3 4 5 -

SyntaxError: invalid syntax (2386377581.py, line 1)